<a href="https://colab.research.google.com/github/abir101-prog/pytorch_experiments/blob/main/autograd_kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

In [5]:
torch.cuda.is_available()

True

In [6]:
torch.cuda.device_count()

1

In [8]:
torch.cuda.get_device_name()

'Tesla T4'

In [10]:
#device
cuda0 = torch.device('cuda:0')

In [21]:
a = torch.tensor([2., 1, 3], requires_grad=True)
a.is_leaf

True

In [25]:
import torch
from torchvision.models import resnet18, ResNet18_Weights
model = resnet18(weights = ResNet18_Weights.DEFAULT)
data = torch.rand(1, 3, 64, 64)
labels = torch.rand(1, 1000)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

# **Forward Pass**

In [26]:
prediction = model(data)

#### Error calc and 
# **Backward pass**

In [28]:
loss = (prediction - labels).sum()
loss

tensor(-513.8189, grad_fn=<SumBackward0>)

In [29]:
loss.backward()

# **Optimizer**

In [30]:
optim = torch.optim.SGD(model.parameters(), lr=1e-2, momentum=0.9)

## initiate gradient descent by ```step()```

In [31]:
optim.step()

just some random autograd testing

In [33]:
a = torch.tensor([1., 2, 3, 4], requires_grad=True)

In [34]:
a

tensor([1., 2., 3., 4.], requires_grad=True)

In [35]:
y = a**2

In [36]:
y

tensor([ 1.,  4.,  9., 16.], grad_fn=<PowBackward0>)

In [38]:
z = sum(y)

In [39]:
z

tensor(30., grad_fn=<AddBackward0>)

In [40]:
z.backward()

In [41]:
z

tensor(30., grad_fn=<AddBackward0>)

In [45]:
a.grad

tensor([2., 4., 6., 8.])

In [47]:
y.grad

/usr/local/lib/python3.8/dist-packages/torch/_tensor.py:1083: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at  aten/src/ATen/core/TensorBody.h:477.)
  return self._grad


In [49]:
y.is_leaf

False

In [50]:
a.is_leaf

True

In [51]:
z.is_leaf

False

the value of gradient is stored in the ```grad``` attribute of the tensor wrt which the grad is taken

https://www.kaggle.com/code/anandsubbu007/pytorch-autograd-tutorial-2/notebook

In [52]:
x = torch.tensor([5], dtype=torch.float32, requires_grad=True)
y = torch.tensor([6], dtype=torch.float32, requires_grad=True)

print(x, y)

tensor([5.], requires_grad=True) tensor([6.], requires_grad=True)


function

In [53]:
z = (x**2)*y + x*y
print(z)

tensor([180.], grad_fn=<AddBackward0>)


In [55]:
print(x.grad, y.grad) # derivative calculated after backward called so now its none

None None


In [56]:
total = torch.sum(z)
total.backward()

In [57]:
print("Def with resp. to  x   :",x.grad)
print("Def with resp. to  y   :",y.grad)

Def with resp. to  x   : tensor([66.])
Def with resp. to  y   : tensor([30.])


# Implementing Autograd

In [65]:
x = torch.randint(-100, 100, (100, ), dtype=torch.float32, device=cuda0)
x # data

tensor([ -45.,  -18.,   39.,   50.,   29.,   96.,   44.,   60.,  -31.,   84.,
         -66.,  -35.,  -48.,  -49.,   77.,  -72.,  -49.,   28.,  -97.,   20.,
         -20.,  -18.,  -87.,  -47.,  -41.,   59.,  -46.,   13.,  -53.,  -17.,
         -13.,  -62.,  -30.,   21.,  -48.,  -62.,  -60.,  -77.,  -55., -100.,
          -8.,   96.,   -3.,   30.,  -77.,  -35.,  -59.,   -4.,   62.,   33.,
         -90.,  -86.,   43.,   61.,  -15.,  -98.,   30.,   87.,  -33.,  -65.,
          66.,  -56.,  -18.,  -33.,  -14.,   97.,   39.,  -97.,   87.,  -54.,
         -68.,   30.,   88.,   67.,   37.,  -67.,  -64.,   -7.,   51.,   63.,
         -18.,  -98.,  -11.,   92.,  -41.,   11.,   55.,   73.,  -66.,  -95.,
         -58.,  -19.,    7.,    7.,   33.,   28.,  -39.,  -69.,  -11.,  -43.],
       device='cuda:0')

In [66]:
y = (1.32 * x) + 25  # prediction

In [75]:
w = torch.ones(1, requires_grad=True, device=cuda0)  # weights and bias
b = torch.ones(1, requires_grad=True, device=cuda0)
y_hat = (w * x) + b

In [76]:
epochs = 10000
lr = 1e-6

In [77]:
count = 0
for i in range(epochs):
  loss = torch.sum((y_hat - y) ** 2)
  loss.backward()  # grad calculated

  with torch.no_grad(): # a context manager to stop from calculating grad anymor.
  # it will improve runtime since requires_grad=false 
  #for everyone inside the context manager
    w -= lr * w.grad
    b -= lr * b.grad
    count += 1
    w.grad.zero_() # setting them 0
    b.grad.zero_()

  y_hat = (w * x) + b 

print(count)
print("Predicted w value  :",w.item())
print("Predicted b value  :",b.item())

10000
Predicted w value  : 1.30953848361969
Predicted b value  : 21.554981231689453


tensor([-158259.2188], device='cuda:0')